### 라이브러리

In [5]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from dotenv import load_dotenv

### API 키

In [6]:
load_dotenv()

# API 키 설정
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

### PDF 로드 / 임베딩

In [7]:
# PDF 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                               chunk_overlap = 20,
                                               )
docs = text_splitter.split_documents(pages)

# OpenAI 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# FAISS 벡터 저장소
vectorstore = FAISS.from_documents(docs, 
                                   embedding=embeddings,
                                   )
retriever = vectorstore.as_retriever()

In [10]:
# Claude LLM
llm = ChatAnthropic(model="claude-3-5-haiku-20241022",
                    temperature=0,
                    max_tokens=1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문
question = "고지의무를 위반한 경우 보험사는 어떤 조치를 취할 수 있나요?"

# 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# 2. 순수 LLM 응답 (문서 없이 바로 Claude에게 질문)
llm_only_answer = llm.invoke(question)

# 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)
print("\n📚 [RAG 기반 응답]:")
print(rag_answer)



C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


🧾 질문: 고지의무를 위반한 경우 보험사는 어떤 조치를 취할 수 있나요?

💬 [LLM 단독 응답]:
content='고지의무 위반 시 보험사가 취할 수 있는 주요 조치는 다음과 같습니다:\n\n1. 계약 해지\n- 고의적 고지의무 위반 시 보험계약 전체를 해지할 수 있음\n\n2. 보험금 지급 거절\n- 고지의무 위반과 보험사고 사이에 인과관계가 있는 경우\n- 보험금 전부 또는 일부를 지급하지 않을 수 있음\n\n3. 계약 내용 변경\n- 보험료 조정\n- 보장 범위 제한\n\n4. 계약 무효 처리\n- 중대한 고지의무 위반 시 계약 자체를 무효화할 수 있음\n\n5. 손해배상 청구\n- 고의적 위반의 경우 추가적인 손해배상 청구 가능' additional_kwargs={} response_metadata={'id': 'msg_01JuHjPbZ7V795EFjinMQaYW', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 43, 'output_tokens': 277, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-haiku-20241022'} id='run--5c80bf91-cc69-425a-b54e-6fc9a693073b-0' usage_metadata={'input_tokens': 43, 'output_tokens': 277, 'total_tokens': 320, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}}

📚 [RAG 기반 응답]:
고지의무 위반 시 보험사는 계약을 해지할 수 있습니다. 구체적으로 보험자는 고의 또는 중대한